In [3]:
import os
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import torch
from torch import nn
import torch.optim as optim
import torchvision
#pip install torchvision
from torchvision import transforms, models, datasets
#https://pytorch.org/docs/stable/torchvision/index.html
# import imageio
import time
import warnings
import random
import sys
import copy
import json
from PIL import Image

In [ ]:
def load_annotations(ann_file):
    data_infos = {}
    with open(ann_file) as f:  #打开文件
        samples = [x.strip().split(' ') for x in f.readlines()] #一行行去读，以空格作为切分，[ [xxx.jpg,'1' ],[xxx,jpg,'2' ]  ]
        for filename, gt_label in samples:  # filename为xxx.jpg，gt_label为分类值
            data_infos[filename] = np.array(gt_label, dtype=np.int64)#构建字典，图片名为key，分类值为value，组合
    return data_infos #返回字典

In [ ]:
img_label = load_annotations('./flower_data/train.txt')
 
image_name = list(img_label.keys())
label = list(img_label.values())

In [ ]:
data_dir = './flower_data/'
train_dir = data_dir + '/train_filelist'
valid_dir = data_dir + '/val_filelist'
 
image_path = [os.path.join(train_dir,img) for img in image_name] #根据图像路径和图像名字加载图片进来

In [ ]:
from torch.utils.data import Dataset, DataLoader
class FlowerDataset(Dataset):  #继承Dataset     生成2个list，一个[包含图片路径]，一个[图像对应的labels]
    def __init__(self, root_dir, ann_file):
        self.ann_file = ann_file
        self.root_dir = root_dir
        self.img_label = self.load_annotations()#返回字典，图片名为key，图片label为value
        self.img = [os.path.join(self.root_dir,img) for img in list(self.img_label.keys())]#一个[包含图片路径]的list 
        self.label = [label for label in list(self.img_label.values())]# [图像对应的labels] 的list
        self.transform = transform
 
    def __len__(self): #数据集样本数量
        return len(self.img)
 # __getitem__会执行 batch_size次，__getitem__返回的数据是给模型的
    def __getitem__(self, idx):#图像和标签在当前list的索引，每次调用idx是随机值，一个batch里的数据是随机打乱的
        image = Image.open(self.img[idx])
        label = self.label[idx]
        label = torch.from_numpy(np.array(label)) #转回Tensor格式
        return image, label #返回实际图像 ，按batch_size大小数据打包好返回
    def load_annotations(self):
        data_infos = {}
        with open(self.ann_file) as f:
            samples = [x.strip().split(' ') for x in f.readlines()]
            for filename, gt_label in samples:
                data_infos[filename] = np.array(gt_label, dtype=np.int64)#将图片名和label组合成字典数据
        return data_infos

In [ ]:
train_dataset = FlowerDataset(root_dir=train_dir, ann_file = './flower_data/train.txt', transform=data_transforms['train'])
val_dataset = FlowerDataset(root_dir=valid_dir, ann_file = './flower_data/val.txt', transform=data_transforms['valid'])
 
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=True)

In [ ]:
iter(train_loader).next()

In [ ]:
dataloaders = {'train':train_loader,'valid':val_loader}
 
model_name = 'resnet'  #可选的比较多 ['resnet', 'alexnet', 'vgg', 'squeezenet', 'densenet', 'inception']
#是否用人家训练好的特征来做
feature_extract = True 
 
train_on_gpu = torch.cuda.is_available()
 
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')
    
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
 
model_ft = models.resnet18() #上面先指定使用resnet，这里再指定使用多少层的模型
 
 
 
#重新定义输出层
num_ftrs = model_ft.fc.in_features #获取输出层的输入
model_ft.fc = nn.Sequential(nn.Linear(num_ftrs, 102))
input_size = 64 #输入图片的大小

In [ ]:
# 优化器设置
optimizer_ft = optim.Adam(model_ft.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)#学习率每7个epoch衰减成原来的1/10
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False, filename='best.pth'):
    since = time.time()
    best_acc = 0
    model.to(device)
 
    val_acc_history = []
    train_acc_history = []
    train_losses = []
    valid_losses = []
    LRs = [optimizer.param_groups[0]['lr']]
 
    best_model_wts = copy.deepcopy(model.state_dict())
 
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
 
        # 训练和验证
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()  # 训练
            else:
                model.eval()   # 验证
 
            running_loss = 0.0
            running_corrects = 0
 
            # 把数据都取个遍
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
 
                # 清零
                optimizer.zero_grad()
                # 只有训练的时候计算和更新梯度
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)
                    #print(loss)
 
                    # 训练阶段更新权重
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
 
                # 计算损失
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
 
            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)
            
            
            time_elapsed = time.time() - since
            print('Time elapsed {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
 
            # 得到最好那次的模型
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                state = {
                  'state_dict': model.state_dict(),#字典里key就是各层的名字，值就是训练好的权重
                  'best_acc': best_acc,
                  'optimizer' : optimizer.state_dict(),#优化器的状态信息
                }
                torch.save(state, filename)
            if phase == 'valid':
                val_acc_history.append(epoch_acc)
                valid_losses.append(epoch_loss)
                scheduler.step(epoch_loss)#学习率衰减
                
            if phase == 'train':
                train_acc_history.append(epoch_acc)
                train_losses.append(epoch_loss)
        
        print('Optimizer learning rate : {:.7f}'.format(optimizer.param_groups[0]['lr']))
        LRs.append(optimizer.param_groups[0]['lr'])
        print()
 
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
 
    # 训练完后用最好的一次当做模型最终的结果,等着一会测试
    model.load_state_dict(best_model_wts)
    return model, val_acc_history, train_acc_history, valid_losses, train_losses, LRs 